# Plenary Protocols

In [1]:
from pathlib import Path
import glob
import numpy as np
import pandas as pd

In [2]:
VERBOSE = True

In [3]:
def show(table):
    with pd.option_context("display.max_colwidth", None):
        display(table)


def preview_lines(filepath, N=5):
    with open(filepath) as temp:
        head = [next(temp) for i in range(N)]
    temp.close()
    return head

### Data Loading
All csv-files in the specified directory will be parsed and concatenated.

In [4]:
path = "plpr-scraper/data/out"
all_files = glob.glob(path + "/*.csv")

df_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, escapechar="\\")
    df_list.append(df)

df = pd.concat(df_list, axis=0, ignore_index=True)
df_prior_shape = df.shape
show(df[:10])

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party
0,2490136,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,0,"Die Sitzung ist eröffnet. Nehmen Sie bitte Platz. - Liebe Kolleginnen und Kollegen, ich begrüße Sie zu unserer Plenarsitzung.\n\nDie Kollegin Dr. Lukrezia Jochimsen hat am 1. März ihren 75. Geburtstag gefeiert und die Kollegin Edelgard Bulmahn einige Tage später ihren 60. Geburtstag. Im Namen des gesamten Hauses möchte ich dazu auch auf diesem Wege noch einmal herzlich gratulieren und alle guten Wünsche übermitteln.",data/txt/17096.txt,chair,NaN
1,2490137,96,17,0,NaN,NaN,NaN,1,Beifall,data/txt/17096.txt,poi,NaN
2,2490138,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,2,Der Kollege Holger Haibach hat mit Wirkung vom 1. März auf seine Mitgliedschaft im Deutschen Bundestag verzichtet. Als seinen Nachfolger begrüße ich den Kollegen Helmut Heiderich.,data/txt/17096.txt,chair,NaN
3,2490139,96,17,0,NaN,NaN,NaN,3,Beifall,data/txt/17096.txt,poi,NaN
4,2490140,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,4,"Ebenso herzlich willkommen heiße ich den Kollegen Ingo Egloff, der als Nachfolger des Kollegen Olaf Scholz die Mitgliedschaft im Deutschen Bundestag erworben hat.",data/txt/17096.txt,chair,NaN
5,2490141,96,17,0,NaN,NaN,NaN,5,Beifall,data/txt/17096.txt,poi,NaN
6,2490142,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,6,"Die CDU/CSU-Fraktion hat mitgeteilt, dass die Kollegin Sibylle Pfeiffer ihr Amt als Schriftführerin niedergelegt hat.",data/txt/17096.txt,chair,NaN
7,2490143,96,17,0,Stefan Müller [Erlangen] [CDU/CSU],Stefan Müller,stefan-muller,7,Dem kann nicht zugestimmt werden!,data/txt/17096.txt,poi,cducsu
8,2490144,96,17,0,Sören Bartol [SPD],Sören Bartol,soren-bartol,8,Wollte sie keine Krawatte tragen?,data/txt/17096.txt,poi,spd
9,2490145,96,17,0,Dr. Dagmar Enkelmann [DIE LINKE],Dr. Dagmar Enkelmann,dagmar-enkelmann,9,Weil sie keine Krawatte trägt?,data/txt/17096.txt,poi,linke


### Data Cleansing
Quick look at the data before cleansing with Pandas-Profiling. We've got 592199 observations, many of which are not speeches but contributions by the *chair* (president and vice-presidents of the parliament) and *POIs*, points of information (interruptions or interjections by MPs other than the speaker or chair). Speeches span multiple rows and require concatenation.

In [5]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pre-Cleansing Pandas Profiling Report")

In [6]:
if VERBOSE:
    profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

For now, we'll keep only speeches and replace newlines read-in literally.

In [7]:
df.dropna(subset=["text"], inplace=True)
df.loc[:, "text"] = df.loc[:, "text"].str.replace("\n\n", " ").replace("\n", " ")
df.loc[:, "text"] = df.loc[:, "text"].astype(str)
df = df[df["type"] == "speech"].copy()
df.reset_index(drop=True, inplace=True)
df.drop(["Unnamed: 0", "row.names"], axis=1, inplace=True, errors="ignore")
print(f"The shape is reduced from {df_prior_shape[0]} rows to {df.shape[0]}")
show(df[["speaker_cleaned", "text"]][:3])

The shape is reduced from 592199 rows to 238648


,speaker_cleaned,text
0,"Dr. Angela Merkel,","Herr Präsident! Meine Damen und Herren! Am Freitag der letzten Woche, 14.45 Uhr Ortszeit, bebte in Japan die Erde. Seismologen maßen eine Stärke von 8,9, später korrigiert auf 9,0. Es war das schwerste Erdbeben in der Geschichte Japans. Sein Epizentrum lag circa 130 Kilometer östlich der Stadt Sendai und circa 400 Kilometer nordöstlich der japanischen Hauptstadt Tokio. Um 16 Uhr Ortszeit desselben Tages traf eine bis zu 10 Meter hohe Flutwelle auf die Ostküste der japanischen Hauptinsel Honshu. Sie richtete schwerste Verwüstungen an. Noch am Abend dieses Tages gab es Meldungen, wonach in einem Reaktor des Kernkraftwerks Fukushima I die Kühlung ausgefallen und im Atomkraftwerk Onagawa ein Feuer ausgebrochen war. Die japanische Regierung rief den atomaren Notstand aus. In den folgenden Tagen und Nächten erschütterten zahlreiche, zum Teil schwere Nachbeben das Land - und das bis heute. Erdbeben und Tsunami haben weite Landstriche von Japans Nordosten verwüstet. Ganze Ortschaften wurden ausgelöscht. Die Zahl der Opfer schnellt seit Tagen in die Höhe. Wie viele es tatsächlich sind - wir wissen es nicht. Zu viele Menschen werden vermisst. Unzählige Häuser und Straßen sind zerstört. Unendlich viele Menschen haben ihr Obdach verloren. Strom wird rationiert oder ist ganz weg. Treibstoff, Trinkwasser, Nahrungsmittel sind knapp. Rund um das Kernkraftwerk Fukushima wurde die Evakuierungszone seit Freitag immer wieder erweitert. Arbeiter dort führen einen ebenso - man kann es nicht anders sagen - heldenhaften wie verzweifelten Kampf gegen den atomaren Super-GAU. Sie setzen dabei nicht nur ihre Gesundheit aufs Spiel, sondern auch ihr Leben ein. Immer dramatischer entwickeln sich die Ereignisse dort: ausgefallene Kühlanlagen, Berichte über freiliegende Brennstäbe, die sich immer stärker erhitzen, Explosionen in verschiedenen Reaktoren, in einem Fall wohl auch mit der Folge der Beschädigung eines Sicherheitsbehälters, Radioaktivität tritt aus. Es ist davon auszugehen, dass es in drei der Anlagen zu schweren Schäden an den Reaktorkernen gekommen ist. Was uns angesichts all dieser Berichte und Bilder, die wir seit letztem Freitag sehen und zu verstehen versuchen, erfüllt, das sind Entsetzen, Fassungslosigkeit, Mitgefühl und Trauer. Die Katastrophe in Japan hat ein geradezu apokalyptisches Ausmaß, und es fehlen die Worte. Unsere tiefste Anteilnahme, unsere Gedanken und unsere Gebete sind bei den Menschen in Japan."
1,"Dr. Angela Merkel,","In dieser Stunde schwerster Prüfung steht Deutschland an der Seite Japans. Was immer wir tun können, um den Menschen in Japan bei der Bewältigung dieser schier unfassbaren Katastrophe zu helfen, das werden wir weiter tun. Das habe ich Premierminister Kan übermittelt, und das hat auch der Bundesaußenminister seinem japanischen Kollegen gesagt. Experten des Technischen Hilfswerks haben in den vergangenen Tagen vor Ort bei der Suche nach Überlebenden geholfen. Ich danke ihnen, und ich danke den Helfern anderer Organisationen für ihren Einsatz für die Menschen in Japan."
2,"Dr. Angela Merkel,","Ich danke allen Helfern des Krisenstabes im Auswärtigen Amt und der Botschaft vor Ort. Sie koordinieren unsere Hilfe. Sie unterstützen auch alle deutschen Staatsangehörigen im Krisengebiet bei einer Ausreise, wenn sie das wünschen. Auch die Vereinten Nationen haben ein Team nach Japan entsandt. Es soll die japanische Regierung dabei unterstützen, die Aufbaumaßnahmen zu koordinieren. Ebenfalls ihre Hilfe angeboten hat die Europäische Union. Liebe Kolleginnen und Kollegen, die Folgen dieser Katastrophe sind überhaupt noch nicht absehbar. Die Betroffenen vor Ort hatten noch fast gar keine Chance, festzustellen, in welchen Bereichen sie tatsächlich weitere Hilfe genau benötigen. Denn der Albtraum immer neuer Beben und nuklearer Horrorszenarien hat noch kein Ende gefunden. In dieser Lage ist es unverzichtbar, dass wir den Menschen in Japan zeigen: Sie sind nicht allein. Dabei zählt di

todo describe what happens

In [8]:
df["previous_speaker_fp"] = df["speaker_fp"].shift(1)
df["new_speaker"] = df["speaker_fp"] != df["previous_speaker_fp"]
df["speech_identifier"] = np.nan
df

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party,previous_speaker_fp,new_speaker,speech_identifier
0,2490153,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,17,Herr Präsident! Meine Damen und Herren! Am Fre...,data/txt/17096.txt,speech,NaN,NaN,True,NaN
1,2490155,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,19,In dieser Stunde schwerster Prüfung steht Deut...,data/txt/17096.txt,speech,NaN,angela-merkel,False,NaN
2,2490157,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,21,Ich danke allen Helfern des Krisenstabes im Au...,data/txt/17096.txt,speech,NaN,angela-merkel,False,NaN
3,2490159,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,23,Das ist Hilfe unter Freunden. Japan war und is...,data/txt/17096.txt,speech,NaN,angela-merkel,False,NaN
4,2490161,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,25,"und ich lehne es auch weiterhin ab, zwar die K...",data/txt/17096.txt,speech,NaN,angela-merkel,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238643,2421414,58,17,0,"Dirk Niebel, Bundesminister für wirtschaftlich...",Dirk Niebel,dirk-niebel,2339,Herr Präsident! Ihre Anmerkung ist vollkommen ...,data/txt/17058.txt,speech,NaN,volkmar-klein,True,NaN
238644,2421424,58,17,0,Dr. Sascha Raabe (SPD),Dr. Sascha Raabe,sascha-raabe,2349,Herr Präsident! Meine Damen und Herren! Es ist...,data/txt/17058.txt,speech,spd,dirk-niebel,True,NaN
238645,2421427,58,17,0,Dr. Sascha Raabe (SPD),Dr. Sascha Raabe,sascha-raabe,2352,"Dann ist es nur richtig, darauf hinzuweisen, d...",data/txt/17058.txt,speech,spd,sascha-raabe,False,NaN
238646,2421429,58,17,0,Dr. Sascha Raabe (SPD),Dr. Sascha Raabe,sascha-raabe,2354,Ich sage einmal: Dem einen oder anderen Mitgli...,data/txt/17058.txt,speech,spd,sascha-raabe,False,NaN


In [9]:
%%time
speech_identifier = int(0)
for index, row in df.iterrows():
    if row["new_speaker"]:
        speech_identifier += 1
    df.at[index, "speech_identifier"] = speech_identifier

CPU times: user 17.8 s, sys: 67.2 ms, total: 17.8 s
Wall time: 17.9 s


In [10]:
df = (
    df.groupby(["speaker_fp", "speech_identifier"], sort=False)
    .agg(
        {
            "id": "count",
            "sitzung": "first",
            "wahlperiode": "first",
            "speaker": "first",
            "speaker_cleaned": "first",
            "sequence": min,
            "text": " ".join,
            "filename": "first",
            "type": "first",
            "speaker_party": "first",
        }
    )
    .reset_index()
)

In [11]:
df.to_pickle("data/plpr.pkl")

### All Text
To model topics, metainformation for speeches is not relevant. Everything in the text column can be glued together for that purpose.

In [12]:
path_alltext = Path("data/plpr_alltext.txt")

df["text"].to_csv(path_alltext, sep=" ", index=False, header=False)
preview_lines(path_alltext, N=2)

['"Herr Präsident! Meine Damen und Herren! Am Freitag der letzten Woche, 14.45 Uhr Ortszeit, bebte in Japan die Erde. Seismologen maßen eine Stärke von 8,9, später korrigiert auf 9,0. Es war das schwerste Erdbeben in der Geschichte Japans. Sein Epizentrum lag circa 130 Kilometer östlich der Stadt Sendai und circa 400 Kilometer nordöstlich der japanischen Hauptstadt Tokio. Um 16 Uhr Ortszeit desselben Tages traf eine bis zu 10 Meter hohe Flutwelle auf die Ostküste der japanischen Hauptinsel Honshu. Sie richtete schwerste Verwüstungen an. Noch am Abend dieses Tages gab es Meldungen, wonach in einem Reaktor des Kernkraftwerks Fukushima I die Kühlung ausgefallen und im Atomkraftwerk Onagawa ein Feuer ausgebrochen war. Die japanische Regierung rief den atomaren Notstand aus. In den folgenden Tagen und Nächten erschütterten zahlreiche, zum Teil schwere Nachbeben das Land - und das bis heute. Erdbeben und Tsunami haben weite Landstriche von Japans Nordosten verwüstet. Ganze Ortschaften wurden